In [1]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools



from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time
import copy


In [ ]:
#BT data loading

# dta_RDD = sc.textFile("file:///home/tguo/data/tian-test/udpjitter-H1april2014_regTree.csv")

dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/udpjitter-H1april2014_regTree_mllib.csv")


dta_splited = dta_RDD.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),int( float(r[1])), 
                                                                 int(float(r[2])),int(float(r[3])),
                                                                 int(float(r[4])),int(float(r[5])),
                                                                 int(float(r[6])),int(float(r[7])) ))
# dta_splited.first()

dta= dta_splited
dta.cache()
print dta.first()
print dta.count()


# dta_filtered = dta.filter(lambda line: line[0] <2000 )
# dta_filtered.cache()
# dta_filtered.count()
# print dta.count()



In [ ]:
#block3: load synthetic data
# dta_RDD = sc.textFile("hdfs://computer61.ant-net/user/tguo/syn-30m-1p10mag.csv",10)

dta_rdd = sc.textFile("file:///home/tguo/data/tian-syn/syn-10m-1p.txt").cache()

print dta_rdd.first()

# data type conversion and organize data of the form (dependent variable value, 
# feature-values) 
dta_splited = dta_rdd.map(lambda line: line.split(",")).map(lambda r:
                                                        ( float(r[5]),
                                                        int(float(r[0])),int( float(r[1])), 
                                                        int(float(r[2])),int(float(r[3])),
                                                        int(float(r[4])),
                                                        int(float(r[5])),
                                                        int(float(r[6])),int(float(r[7])),
                                                        int(float(r[8]))
                                                         ,int(float(r[9])),
                                                        ) )


dta= dta_splited
print dta.first()
print dta.count()
print dta.getNumPartitions()

In [ ]:
#  load flight data

dta_rdd = sc.textFile("hdfs://computer61.ant-net/user/tguo/airline_data.csv")

dta = dta_rdd.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),
                                                                 int( float(r[1])), 
                                                                 int(float(r[2])),
                                                                 int(float(r[3])),
                                                                 int(float(r[4])),
                                                                 int(float(r[5])),
                                                                 int(float(r[6])),
                                                                 int(float(r[7])), 
                                                                 int(float(r[8])))).cache()


print dta.first()
print dta.count()

tmprdd =  dta.map(lambda line:line[0])
print tmprdd.max(), tmprdd.min()


In [4]:
number_workers=5
maxdepth=10
numFeatures=10
trimm_ratio = 0.00
bin_num=500

In [7]:
# basic functions in one-pass method  [on the cluster side]

# assign each data instance to the node
def search_nodeToData(features, tree):
    
    nodeNum=len(tree)   
    if nodeNum == 0:
        return 0;
    
    current_nodeIdx=0
    while current_nodeIdx< nodeNum:
        split_feature= tree[current_nodeIdx][0]
        split_valueSet= tree[current_nodeIdx][1]
        
        # unbalanced tree grow
        if split_feature==-1:
            current_nodeIdx=current_nodeIdx*2+1+ random.randint(0, 1)
            continue
            
        if (( 1<<features[split_feature]) &  split_valueSet) != 0 :
            current_nodeIdx=current_nodeIdx*2+1
        else:
            current_nodeIdx=current_nodeIdx*2+2
            
    return current_nodeIdx - nodeNum

def update_hist(hist, newY):

    numbins=len(hist)
    
    if newY in hist.keys():
        hist[newY][2]= hist[newY][2] +1 
    else:
        hist.update({newY: [newY,newY,1]})
        if numbins+1 > bin_num:
            hist.update({newY: [newY,newY,1]}) 
            #data in each bin: left bound, right bound, count
        
            sorted_binIds=hist.keys()
            # binId is the lower bound of the value range of the bin
            sorted_binIds.sort()
        
            mindis= sorted_binIds[1]- sorted_binIds[0]
            mindis_bin_left=sorted_binIds[0]
            mindis_bin_right=sorted_binIds[1]
            
            for i in range(1,numbins+1):
                tmp= sorted_binIds[i]- sorted_binIds[i-1]
                if tmp < mindis:
                    mindis=tmp
                    mindis_bin_left= sorted_binIds[i-1]
                    mindis_bin_right= sorted_binIds[i]       
            
#           print hist_local[mindis_bin_right][1],hist_local[mindis_bin_left][1]
            hist[mindis_bin_left][1] = max(hist[mindis_bin_right][1],hist[mindis_bin_left][1] )                        
            hist[mindis_bin_left][2] = hist[mindis_bin_left][2] + \
            hist[mindis_bin_right][2]
            del hist[mindis_bin_right]
            
    return hist

def merge_hist(hist1, hist2):
    tmp=[]
    concat_hist=hist1
    
    if len(hist1)==0:
        return hist2
    if len(hist2)==0:
        return hist1

    for i in concat_hist.keys():
        if i in hist2.keys():
            concat_hist.update( {i: [i,max( concat_hist[i][1], hist2[i][1]),concat_hist[i][2]+hist2[i][2]]})
    
    for i in hist2.keys():
        if i not in concat_hist.keys():
            concat_hist.update( {i:hist2[i]})

    cnt_total= len(concat_hist)
    if cnt_total <= bin_num:
        return concat_hist
    else:
        bins=concat_hist.keys()
        bins.sort()
        
        disl=[]
        disr=[]
        tmpdis=0
        tmpleft=0
        tmpright=0
        
        for i in range(0,cnt_total-1):
            tmpleft= bins[i]
            tmpright= bins[i+1]
            tmpdis= tmpright-tmpleft 
            disl.append((tmpdis,tmpleft))
            disr.append((tmpdis,tmpright))
        disl.sort()
        disr.sort()
        
        bin_num_toRemove=cnt_total - bin_num
        disIdx=[]
        cur=0
        
        for i in range(0,bin_num_toRemove):
            while disl[cur][1] not in concat_hist.keys()  :
                cur=cur+1
            
            tmphist1= concat_hist[disl[cur][1] ]
            tmphist2= concat_hist[disr[cur][1] ]
            tmpval1=tmphist1[1]
            tmpval2=tmphist2[1]
            tmphist1[1]= max(tmpval1, tmpval2)
            tmphist1[2]=  tmphist1[2] + tmphist2[2]
            del concat_hist[disr[cur][1]]
            
            cur=cur+1
            
    return concat_hist

def partition_combiner_hist(list_dvAndfeatures):
    
    nodes_dict={}
    tmpcnt=0
    
#     #debug
#     for line in dvFeatures:
#         tmp_var= line[0]
    
    
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        
        if node in nodes_dict.keys():
            
            # new added: s um of y in a node
            nodes_dict[node]['sumY'] = nodes_dict[node]['sumY']+Y 
#             nodes_dict[node]['sumSquY'] = nodes_dict[node]['sumSquY']+Y*Y 
            nodes_dict[node]['count'] = nodes_dict[node]['count']+1 
            nodes_dict[node]['hist']= update_hist(nodes_dict[node]['hist'], Y)
            
            for i in range(0,numFeatures):    
                feature_val= dvAndfeatures[i+1]
                
                if feature_val not in nodes_dict[node][i]:
                    nodes_dict[node][i].update( { feature_val: {}   }  )
    
                    nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                    nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )
                    nodes_dict[node][i][feature_val]['hist_inFeatureValue'].update({Y:[Y,Y,1]})
                    
                else:
                    nodes_dict[node][i][feature_val]['count_inFeatureValue'] = \
                    nodes_dict[node][i][feature_val]['count_inFeatureValue']+1 
                    nodes_dict[node][i][feature_val]['hist_inFeatureValue']=\
                    update_hist(nodes_dict[node][i][feature_val]['hist_inFeatureValue'], Y)
                    
            
        else:
            nodes_dict.update( {node: {}})  
            
            # new added: sum of y in a node
            nodes_dict[node].update( { 'sumY': Y} )
            nodes_dict[node].update( { 'count': 1} )
            nodes_dict[node].update( { 'hist': {}} )
            nodes_dict[node]['hist'].update({Y:[Y,Y,1]})            
        
            for i in range(0,numFeatures):
                feature_val= dvAndfeatures[i+1]
                
                nodes_dict[node].update( { i: {}} )    
                nodes_dict[node][i].update( { feature_val: {}   }  )
             
                nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )
                nodes_dict[node][i][feature_val]['hist_inFeatureValue'].update({Y:[Y,Y,1]})    
                        
    return zip(nodes_dict.keys(), nodes_dict.values())





# def partition_combiner_hist():
    

def partition_combiner_hist_opt(list_dvAndfeatures):
    
    nodes_dict={}
    tmpcnt=0

#     local_dvFeatures=[]        
#     for dvAndfeatures in list_dvAndfeatures:
#         local_dvFeatures.append(dvAndfeatures)
#     local_dvFeatures.sort()
    
    
    
    for dvAndfeatures in list_dvAndfeatures:
        
        Y= dvAndfeatures[0]
        
        node= search_nodeToData(dvAndfeatures[1:numFeatures+1], \
                                currentNode_split_fromMaster.value)
        
        if node in nodes_dict.keys():
            
            # new added: s um of y in a node
            nodes_dict[node]['sumY'] = nodes_dict[node]['sumY']+Y 
#             nodes_dict[node]['sumSquY'] = nodes_dict[node]['sumSquY']+Y*Y 
            nodes_dict[node]['count'] = nodes_dict[node]['count']+1 
            nodes_dict[node]['hist']= update_hist(nodes_dict[node]['hist'], Y)
            
            for i in range(0,numFeatures):    
                feature_val= dvAndfeatures[i+1]
                
                if feature_val not in nodes_dict[node][i]:
                    nodes_dict[node][i].update( { feature_val: {}   }  )
    
                    nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                    nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )
                    nodes_dict[node][i][feature_val]['hist_inFeatureValue'].update({Y:[Y,Y,1]})
                    
                else:
                    nodes_dict[node][i][feature_val]['count_inFeatureValue'] = \
                    nodes_dict[node][i][feature_val]['count_inFeatureValue']+1 
                    nodes_dict[node][i][feature_val]['hist_inFeatureValue']=\
                    update_hist(nodes_dict[node][i][feature_val]['hist_inFeatureValue'], Y)
                    
            
        else:
            nodes_dict.update( {node: {}})  
            
            # new added: sum of y in a node
            nodes_dict[node].update( { 'sumY': Y} )
            nodes_dict[node].update( { 'count': 1} )
            nodes_dict[node].update( { 'hist': {}} )
            nodes_dict[node]['hist'].update({Y:[Y,Y,1]})            
        
            for i in range(0,numFeatures):
                feature_val= dvAndfeatures[i+1]
                
                nodes_dict[node].update( { i: {}} )    
                nodes_dict[node][i].update( { feature_val: {}   }  )
             
                nodes_dict[node][i][feature_val].update( {'count_inFeatureValue':1} )
                nodes_dict[node][i][feature_val].update( {'hist_inFeatureValue': {} } )
                nodes_dict[node][i][feature_val]['hist_inFeatureValue'].update({Y:[Y,Y,1]})    
                        
    return zip(nodes_dict.keys(), nodes_dict.values())

def merge_parttion_combiner_hist(nodeToFeatureToValue_1,nodeToFeatureToValue_2 ):
# optimization: calculate median and amd for feature-value    
    
    # new added: sum of Y in a node
    nodeToFeatureToValue_1['sumY']= nodeToFeatureToValue_1['sumY']+\
    nodeToFeatureToValue_2['sumY']
    
    nodeToFeatureToValue_1['count']= nodeToFeatureToValue_1['count']+\
    nodeToFeatureToValue_2['count'] 
    
    nodeToFeatureToValue_1['hist']=merge_hist(nodeToFeatureToValue_1['hist'],nodeToFeatureToValue_2['hist'])
    
    for i in range(0, numFeatures): #feature
        for j in nodeToFeatureToValue_1[i].keys(): #feature value
            
            feature_val=j
            
            if feature_val in nodeToFeatureToValue_2[i].keys():               
                nodeToFeatureToValue_1[i][j]['hist_inFeatureValue'] = \
                merge_hist(nodeToFeatureToValue_1[i][j]['hist_inFeatureValue'], 
                           nodeToFeatureToValue_2[i][j]['hist_inFeatureValue'])     
                nodeToFeatureToValue_1[i][j]['count_inFeatureValue'] = nodeToFeatureToValue_1[i][j]['count_inFeatureValue']+nodeToFeatureToValue_2[i][j]['count_inFeatureValue']
                
    for i in range(0, numFeatures):
        for j in nodeToFeatureToValue_2[i].keys():
            
            feature_val=j
            
            if feature_val not in nodeToFeatureToValue_1[i].keys():
                nodeToFeatureToValue_1[i].update({feature_val: {} })
                nodeToFeatureToValue_1[i][feature_val]= nodeToFeatureToValue_2[i][feature_val].copy()
                              
    return  nodeToFeatureToValue_1

In [8]:
# split decision  [on the local side]

#MAD: median absolute median in a histogram   
def MAD_hist(hist, cnt): 

    sorted_his= sorted(hist.items(), key= lambda line: line[0])
    num_bins= len(hist)
    
    cntByBounds=0
    curCnt=0 
    midCnt= cnt/2.0
    
    mid_bin_idx=0
    flag=0
    
    meanAbsDevi=0
    median=0
    
    cntSum_beforeMid=0
    
    for i in range(0, num_bins): 
        
        if flag==0:
            tmp_curCnt=curCnt+ sorted_his[i][1][2]
            tmp_cntByBounds =cntByBounds - \
        (sorted_his[i][1][0]+sorted_his[i][1][1])/2.0*sorted_his[i][1][2]
        
            if tmp_curCnt >= midCnt:
                flag=1       
                
                mid_bin_idx= i
                
                cntSum_beforeMid = curCnt
            else:    
                curCnt=tmp_curCnt
                cntByBounds=tmp_cntByBounds
        else:
            curCnt= curCnt - sorted_his[i][1][2]
            cntByBounds= cntByBounds + \
            (sorted_his[i][1][0]+sorted_his[i][1][1])/2.0*sorted_his[i][1][2]
    
    if cnt == 1:
        for tmpkey in hist.keys():
            return (hist[tmpkey][0],0)
    elif cnt ==0:
        return (0,0)
    else:        

        sample_inMedBin = midCnt - cntSum_beforeMid     
        
        if sorted_his[mid_bin_idx][1][2] ==1:
            median = sorted_his[mid_bin_idx][1][0]
        elif sorted_his[mid_bin_idx][1][2] ==2:
            median= (sorted_his[mid_bin_idx][1][0] +sorted_his[mid_bin_idx][1][1])*1.0/2.0
        else:
            median= sorted_his[mid_bin_idx][1][0]*1.0 + \
        (1.0*sorted_his[mid_bin_idx][1][1]-1.0*sorted_his[mid_bin_idx][1][0])/(sorted_his[mid_bin_idx][1][2]+1)*sample_inMedBin
        
        meanAbsDevi =  cntByBounds + curCnt*median 
        
        
        
        if sorted_his[mid_bin_idx][1][2] ==1:
            return (median, meanAbsDevi*1.0/cnt)
        
        sample_val= sorted_his[ mid_bin_idx][1][0]*1.0 
        sample_interval=1.0*(sorted_his[mid_bin_idx][1][1]-sorted_his[mid_bin_idx][1][0])/(sorted_his[mid_bin_idx][1][2]\
                                                                          -1.0)
#         print meanAbsDevi     
    
        for i in range(0, sorted_his[mid_bin_idx][1][2]):
            sample_val= sample_val + sample_interval*i
            meanAbsDevi=meanAbsDevi+ abs( sample_val-median)
            
#             print abs( sample_val-median)
                        
        #debug
#         if meanAbsDevi <0:
#             tmpsum=0
#             for tmpkey in hist:
#                 tmpsum= tmpsum+ hist[tmpkey][2]
#             if tmpsum!=cnt:
#                 print 'inside MAD calculation, number of data check:',tmpsum,cnt             
#             print 'inside MAD calculation:', tmpMAD, cnt
            
        
        return (median, meanAbsDevi*1.0/cnt)
    
    

def local_merge_hist(hist1, hist2):
    
    local_hist= copy.deepcopy(hist1)
    
    for i in hist2.keys():
        if i in local_hist.keys():
            local_hist[i][2] = local_hist[i][2] +hist2[i][2]
            local_hist[i][1]= max(local_hist[i][1], hist2[i][1])
        else:
            local_hist.update( {i: hist2[i]}  )
    return local_hist
    

def split_onOneFeature_hist( values_hist,node_data_cnt, node_hist):
    
    left_count=0
    right_count= node_data_cnt
    
#     if len(values_hist)<=1:
#         return [-1,-1,-1,-1]

    #this is a bit-record variable
    leftSplit_valueSet = 0 
    leftSplit_count = 0
    leftSplit_hist= {}
    rightSplit_hist={}
        
    bestLeft=(-1,-1)
    bestRight=(-1,-1)  
    bestSplitMetric =0 
    
    
    #sort feature values according to median 
    sorted_value_map=[]
    
    for i in values_hist.keys():
        currentVal=i
        currentVal_hist_count= values_hist[i]['count_inFeatureValue']
        currentVal_hist= values_hist[i]['hist_inFeatureValue']
        
        #debug
        if len(currentVal_hist) > bin_num:
            print '!!! histogrm bin number error !!!'
        
        sorted_value_map.append((currentVal,MAD_hist(currentVal_hist,currentVal_hist_count)))
            
    sorted_value_map=sorted(sorted_value_map, key= lambda val: val[1][0] )
    values_cnt= len(sorted_value_map)
    
    #--------- initialize the value set ----------------------
    
    # check the number of feature values
    if values_cnt <= 1:
        return [-1,-1,-1,-1]
    
    current_feature_value = sorted_value_map[0][0] 
    currentVal_count= values_hist[current_feature_value]['count_inFeatureValue']
    currentVal_hist= values_hist[current_feature_value]['hist_inFeatureValue']
    
    #debug
#     if len(currentVal_hist) > bin_num:
#         print '!!! histogrm bin number error !!!'

    leftSplit_count = leftSplit_count+currentVal_count     
    leftSplit_hist=local_merge_hist( leftSplit_hist, currentVal_hist)
    
    # histograms for the right values
    rightSplit_hist={}
    tmp_value=0
    for j in range(0, values_cnt-1 ):
        tmp_value = sorted_value_map[values_cnt-1-j][0]
        rightSplit_hist= local_merge_hist( rightSplit_hist, values_hist[tmp_value]['hist_inFeatureValue'])      
            
    left = MAD_hist( leftSplit_hist, leftSplit_count)
    right= MAD_hist( rightSplit_hist, (node_data_cnt - leftSplit_count)) 
        
    #debug
#     tmpcnt=0
#     for i in leftSplit_hist.keys():
#         tmpcnt= tmpcnt+ leftSplit_hist[i][2]
#     if tmpcnt!= leftSplit_count:
#         print '++++ problem in left split histogram', tmpcnt, leftSplit_count
#     tmpcnt=0
#     for i in rightSplit_hist.keys():
#         tmpcnt= tmpcnt+ rightSplit_hist[i][2]
#     if tmpcnt!= (node_data_cnt - leftSplit_count):
#         print '???? problem in right split histogram', tmpcnt, (node_data_cnt - leftSplit_count)
    
    
    leftMedian= left[0]
    leftMetric= left[1]
    rightMedian= right[0]
    rightMetric= right[1]
        
# #         debug
#     if leftMedian<0 or rightMedian<0 or leftMetric <=0 or rightMetric <=0:
#         print '$$$$ problem in MAD calculation',leftMedian,rightMedian,leftMetric,rightMetric,\
#         leftSplit_count,(node_data_cnt - leftSplit_count),\
#         current_feature_value
            
    current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
    bestSplitMetric=current_splitMetric     
    bestLeft=left
    bestRight=right
    #bit variable to record which value is chosen for the left child
    leftSplit_valueSet=leftSplit_valueSet+ (1<<current_feature_value)
    

    
    split_pos=0
    
    
    #---------------------------------------------------------------

    # scan the sorted feature values
    for k in range(1,values_cnt-1):

        current_feature_value=sorted_value_map[k][0]
        currentVal_count= values_hist[current_feature_value]['count_inFeatureValue']
        currentVal_hist= values_hist[current_feature_value]['hist_inFeatureValue']
        
        
        #debug
#         if len(currentVal_hist) > bin_num:
#             print '!!! histogrm bin number error !!!'
        

#         histograms for the left values
        leftSplit_count = leftSplit_count+currentVal_count     
        leftSplit_hist=local_merge_hist( leftSplit_hist, currentVal_hist)
    
#         histograms for the right values
        rightSplit_hist={}
        tmp_value=0
        for j in range(0, values_cnt-1-k ):
            tmp_value = sorted_value_map[values_cnt-1-j][0]
            rightSplit_hist= local_merge_hist( rightSplit_hist, values_hist[tmp_value]['hist_inFeatureValue'])      
            
        left = MAD_hist( leftSplit_hist, leftSplit_count)
        right= MAD_hist( rightSplit_hist, (node_data_cnt - leftSplit_count)) 
        
        #debug
#         tmpcnt=0
#         for i in leftSplit_hist.keys():
#             tmpcnt= tmpcnt+ leftSplit_hist[i][2]
#         if tmpcnt!= leftSplit_count:
#             print '++++ problem in left split histogram', tmpcnt, leftSplit_count
#         tmpcnt=0
#         for i in rightSplit_hist.keys():
#             tmpcnt= tmpcnt+ rightSplit_hist[i][2]
#         if tmpcnt!= (node_data_cnt - leftSplit_count):
#             print '???? problem in right split histogram', tmpcnt, (node_data_cnt - leftSplit_count)
                 
        leftMedian= left[0]
        leftMetric= left[1]
        rightMedian= right[0]
        rightMetric= right[1]
        
#         #debug
#         if leftMedian<0 or rightMedian<0 or leftMetric <=0 or rightMetric <=0:
#             print '$$$$ problem in MAD calculation',leftMedian,rightMedian,leftMetric,rightMetric,\
#             leftSplit_count,(node_data_cnt - leftSplit_count),\
#             current_feature_value
            
        current_splitMetric=1.0*leftSplit_count/node_data_cnt*leftMetric + \
                               1.0*(node_data_cnt - leftSplit_count)/node_data_cnt*rightMetric
        
        if current_splitMetric < bestSplitMetric:
            
            bestSplitMetric=current_splitMetric
            bestLeft=left
            bestRight=right
            split_pos = k
            
    for k in range(1, split_pos+1):
        leftSplit_valueSet=leftSplit_valueSet+ (1<<sorted_value_map[k][0])        
    
    return [leftSplit_valueSet, bestLeft[0], bestRight[0], bestSplitMetric ]


def find_bestSplit_hist(local_aggre_nodes, current_NumNodes, nodes_tree, \
                        nodes_tree_test):
        
    #debug 
#     tmpnode_cnt=[]
#     presplit=[]
    
    local_node_idx=0
    
#------------- grow the unbalanced tree------------------------    
    cur_node_num= len(nodes_tree)

    for i in range(0, current_NumNodes):

        #------------- grow the unbalanced tree ---------------------------------- 
        if cur_node_num >= 3:
            parent_node = int(math.ceil(  (i + cur_node_num -2.0)*1.0/ 2 ))
            parent_node_esti =nodes_tree_test[2* parent_node +1 ] 
            tmp_split_feature= nodes_tree[parent_node][0]
            
            if tmp_split_feature == -1:
                nodes_tree.append(  (-1,  -1  )  )
                nodes_tree_test.append( parent_node_esti  )
                nodes_tree_test.append( parent_node_esti )
                continue
            else:
                while i!= local_aggre_nodes[local_node_idx][0]:
                    local_node_idx=local_node_idx+1
                    
#                 print '!!!  possible problem !!!!'
        #-------------------------------------------------------------------------
        
        
#         if local_aggre_nodes[ current_idx_nodes ][0] != i:
#             nodes_layer.append( (-1,-1) )
#             nodes_tree.append( (-1,-1) )
#             continue
 
        node_count= local_aggre_nodes[local_node_idx][1]['count'] 
        node_hist=  local_aggre_nodes[local_node_idx][1]['hist']           
        
#         (median, meanAbsDevi)
        
#      ????
        tmp_metric= MAD_hist( node_hist, node_count)
        best_split_sofar= tmp_metric[1]
        best_split=(-1,-1,-1,-1)
        
        #debug
#         tmpnode_cnt.append(node_count)
#         presplit.append( best_splitMetric_feature[1]/100000 )
         
        best_split_feature=-1
        best_split_featureValueSet=-1

        for j in range(0,numFeatures):
                     
            statisticToValues = local_aggre_nodes[local_node_idx][1][j]

            split=split_onOneFeature_hist(statisticToValues,node_count, node_hist)
            
            #debug
#             print split[1],split[2]
#             [leftSplit_valueSet, bestLeft[0],bestRight[0], bestSplitMetric ]

            #debug
#             if len(nodes_tree) ==0:
#                 print 'bug check: ', split
    
            if  split[0]!=-1 and  split[3] < best_split_sofar:
                best_split=split
                best_split_sofar=split[3]
                best_split_feature = j
                best_split_featureValueSet= split[0]
         
        
        local_node_idx=local_node_idx+1
        
        #------------- grow the unbalanced tree ---------------------------------- 
        if best_split_feature == -1:
            nodes_tree.append(  (-1,  -1  )  )
            nodes_tree_test.append( tmp_metric[0]  )
            nodes_tree_test.append( tmp_metric[0] )
            #debug
#             print 'chosen:', tmp_metric[0],tmp_metric[0]
            continue
        #-------------------------------------------------------------------------        
   
        #debug 
#         if len(nodes_tree) ==217:
#             print '--- debug---', best_split_feature,  best_split_featureValueSet,\
#             (local_aggre_nodes[i][1][0].keys()),(local_aggre_nodes[i][1][1].keys()),\
#             (local_aggre_nodes[i][1][2].keys())
            
#         if len(nodes_tree) == 435:
#             print '--- debug---', best_split_feature,  best_split_featureValueSet,\
#             (local_aggre_nodes[i][1][0].keys()),(local_aggre_nodes[i][1][1].keys()),\
#             (local_aggre_nodes[i][1][2].keys())
            
#         if len(nodes_tree) == 872:
#             print '--- debug---', best_split_feature,  best_split_featureValueSet,\
#             (local_aggre_nodes[i][1][0].keys()),(local_aggre_nodes[i][1][1].keys()),\
#             (local_aggre_nodes[i][1][2].keys())

        #debug
#         if current_NumNodes == 512:
#             print '----- debug ----', len(nodes_tree), best_split_feature,best_split_featureValueSet,(local_aggre_nodes[i][1][0].keys()),\
#             (local_aggre_nodes[i][1][1].keys()),(local_aggre_nodes[i][1][2].keys())
#             if len(local_aggre_nodes[i][1][0])==1 and len(local_aggre_nodes[i][1][1])==1 and len(local_aggre_nodes[i][1][2])==1:
#                 print '-------------',local_aggre_nodes[i][1][0].keys(),local_aggre_nodes[i][1][1].keys(),local_aggre_nodes[i][1][2].keys(),\
#                 'split:', best_split_feature,  best_split_featureValueSet

        #debug
#         print 'chosen:', best_split[1],best_split[2]

        # split on each node
        nodes_tree.append(  (best_split_feature,  best_split_featureValueSet  )      )
        
        # tree for predicting
        nodes_tree_test.append(best_split[1])
        nodes_tree_test.append(best_split[2])
    
#     
#     if local_node_idx != len(local_aggre_nodes):
#         print '!!!! debug !!!!: not all local nodes are processed ', local_node_idx,len(local_aggre_nodes)
        
        
        
        

In [9]:
#statistic_infor_check for distributed information collection

def statistic_infor_check(local_aggre_nodes):
    
    numNode = len(local_aggre_nodes)
#     print numNode
    tmpsum=0
    for i in range(0, numNode):
        tmpsum=tmpsum+local_aggre_nodes[i][1]['count']
        
#     print tmpsum
    if tmpsum != 98974:
        print 'problem in node count'

    tmpsum=0
    for i in range(0, numNode):
        #check node hist
        node_count = local_aggre_nodes[i][1]['count']
        tmpsum=0
        for j in local_aggre_nodes[i][1]['hist'].keys():
            tmpsum= tmpsum+local_aggre_nodes[i][1]['hist'][j][2]
        if tmpsum!= node_count:
            print '$$$$ problem in the histogram of node'
            
        #check feature values
        for j in range(0, numFeatures):
            tmpsum=0
            for k in local_aggre_nodes[i][1][j].keys():
                value_hist= local_aggre_nodes[i][1][j][k]['hist_inFeatureValue']
                value_cnt = local_aggre_nodes[i][1][j][k]['count_inFeatureValue']
                tmpsum= tmpsum+value_cnt
            
#                 print len(value_hist)
                if len(value_hist) > bin_num:
                    print '???? bin num wrong', len(value_hist)
            
                #check feature-value hist
                tmpsum1=0
                for m in value_hist.keys():
                    tmpsum1= tmpsum1 + value_hist[m][2]
                if tmpsum1 != value_cnt:
                    print '++++ problem in feature-value histogram', tmpsum1, value_cnt
            
            if tmpsum != node_count:
                print '---- problem !!!! in feature:', tmpsum, node_count    


In [ ]:

# def tree_test_mapFunc_median(line):
#     tmpnode = search_nodeToData(line[1:numFeatures+1], node_split)
#     return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])

# def tree_test_mapFunc(line):
#     tmpnode = search_nodeToData(line[1:numFeatures+1], node_split)
#     return (leaf_nodes[ tmpnode ]  - line[0])*(leaf_nodes[ tmpnode ]  - line[0])
    
# def tree_test( testData_rdd ):
#     err_rdd=testData_rdd.map( lambda line: tree_test_mapFunc(line))
#     err_sum = err_rdd.reduce( lambda a,b: a+b)
#     return err_sum / testData_rdd.count()


dta_train= dta_train.repartition(number_workers)
print dta_train.getNumPartitions()
print dta_train.first()

# variables for the tree structure 
node_split=[]
node_test=[]

test_err = []
train_err = []
run_time=[]

ltime=[]
ctime=[]

tree_history_split=[]
tree_history_esti=[]
tree_history_leaf=[]
tree_history_runtime=[]

#ini parameter on the cluster
currentNode_split_fromMaster = sc.broadcast([])

#prepare feature value set
# feature_valueSet = data_featureValues_collect( dta_train )
# feature_valueList=[]
# for i in range(0, len(feature_valueSet)):
#     feature_valueList.append( ( i, list(feature_valueSet[i][1]) ) )

#     tune the starting depth
for i in range(maxdepth,maxdepth+1):
    
    del node_split[:]
    del node_test[:]
    currentNode_split_fromMaster = sc.broadcast([])
    
    for cur_depth in range(0,i):
        
        start_cluster = time.time() 
        
        current_NumNodes= int( math.pow(2,  cur_depth)) 
        statis_partitions = dta_train.mapPartitions( partition_combiner_hist )
        
        aggre_nodes =statis_partitions.reduceByKey(lambda statis_partition_1, statis_partition_2: 
                              merge_parttion_combiner_hist(statis_partition_1,statis_partition_2 ))
        
        local_aggre_nodes= aggre_nodes.collect()
        
        #debug
#         statistic_infor_check(local_aggre_nodes)

        #debug
#         print len(local_aggre_nodes) , current_NumNodes
        
        end_cluster = time.time()
        
        find_bestSplit_hist( sorted(local_aggre_nodes,key= lambda val:val[0])\
                                    ,current_NumNodes, node_split, node_test)
        end_local= time.time()
        
        #debug
#         statistic_infor_check(local_aggre_nodes)
    
        ctime.append(  end_cluster- start_cluster )
        ltime.append(  end_local - end_cluster )
    
        currentNode_split_fromMaster = sc.broadcast(node_split)
        print "time consumption ratio", i,":",  end_cluster-start_cluster, end_local-end_cluster
    
    
    leaf_nodes = node_test[ len(node_test)-  (int)( math.pow(2,i)): len(node_test)  ]
    
    tmp_test_err= 0 
    tmp_train_err= 0

    tree_history_split.append(copy.deepcopy(node_split) )
    tree_history_esti.append( copy.deepcopy(node_test)  )
    tree_history_leaf.append( copy.deepcopy(leaf_nodes)  )
        
    
    print ctime
    print ltime
    

In [18]:
# training and testing process: extract all the feature-value combinations 
# in the training data set

total_featureVal_set=[]
for i in range(0, numFeatures):
    featureValues = dta_train.map(lambda line: line[1+i]).distinct().collect()
    total_featureVal_set.append( featureValues)

print total_featureVal_set    
    
print 'done'


[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]
done


In [ ]:
# LAD result statistic : test error

#---------------- LAD -------------------  data backup
lad_tree_history_split = copy.deepcopy( tree_history_split)
lad_tree_history_esti =  copy.deepcopy( tree_history_esti)
lad_tree_history_leaf =  copy.deepcopy( tree_history_leaf)
lad_tree_history_runtime =  copy.deepcopy( tree_history_runtime)

# print len(lad_tree_history_split)
# print len(lad_tree_history_split[0])
# print len(lad_tree_history_esti)
# print len(lad_tree_history_esti[0])

# print len(lad_tree_history_leaf)
# print len(lad_tree_history_leaf[0])


pre=0
tree_esti = lad_tree_history_esti[0]
tree_split =  lad_tree_history_split[ 0 ]
err_depth=[]

for i in range(1,maxdepth+1):
            
    end_bound= pre+ (int)( math.pow(2,i)) 
    test_leaf_nodes = tree_esti[ pre: end_bound  ]
    pre= end_bound        
    test_tree_split= tree_split[ 0:(int)( math.pow(2,i)-1)  ]
    
#     print len(test_tree_split)
#     print len(test_leaf_nodes)

    err_depth.append(test_error_tree( i, dta_test)  ) 
    

print err_depth

In [13]:
# data inspection for the nodes of the trained tree

def feature_value_extract( valueSet):
    
    tmpval=1
    tmp_valSet= valueSet
    valSet_list=[]
    cnt=0
    
    while tmpval <= tmp_valSet:
        if tmpval & tmp_valSet !=0:
            valSet_list.append(cnt)
        tmpval=tmpval<<1
        cnt=cnt+1
        
    return valSet_list

#????
def training_data_summary( train_rdd ):

#     always remove outliers before
    train_rdd_clean= train_rdd.filter(lambda line: line[0]<1000)
    distinct_featureVals = train_rdd_clean.map(lambda line: (line[1], line[2],line[3],line[4] ) ).distinct().collect()
   
    featureConfig_val={}
    for i in distinct_featureVals:
        
        tmprdd=train_rdd_clean.filter(lambda line:(line[1], line[2],line[3],line[4])  ==i ).map(lambda line: line[0])
        tmpval=tmprdd.reduce(lambda a,b:a+b)
        tmpval= tmpval/tmprdd.count()*1.0
        
        featureConfig_val.update( {i:tmpval} )
    
#     print featureConfig_val
    return featureConfig_val
        

def config_check( featureVal_set, feature_num):
    
    
    for i in range(0, feature_num):
        if len(featureVal_set[i]) >1 :
            return -1
    return 1

def lookup_featureConfig_val( configToVal_dict, value_set, feature_num):
    
    tmptuple=()
    for i in range(0, feature_num):
        tmptuple = tmptuple + ( value_set[i][0]  ,)
#     print '++++++++++++', tmptuple
    return configToVal_dict[tmptuple]



def test_error_indiviMapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], test_tree_split)
    return (test_leaf_nodes[ tmpnode ], line[0])
    
def test_error_mapFunc(line):
    tmpnode = search_nodeToData(line[1:numFeatures+1], test_tree_split)
    
#     return ( test_leaf_nodes[ tmpnode ],  line[0]   )
    return (test_leaf_nodes[ tmpnode ]  - line[0])*(test_leaf_nodes[ tmpnode ]  - line[0])
    
def test_error_tree( current_depth, testRDD  ):
    
    test_cnt= testRDD.count()
    err_rdd=testRDD.map( lambda line: test_error_mapFunc(line))
    err_sum = err_rdd.reduce( lambda a,b: a+b)
    
    print 'test error at depth',current_depth, ':', sqrt(err_sum*1.0/test_cnt)/(ymax-ymin)*1.0
    return sqrt(err_sum*1.0/test_cnt)/(ymax-ymin)*1.0
    
def bfs_tree(tree, total_featureVal_set, depth, tree_esti, trainRDD ):
    
    nodeNum=len(tree)   
    if nodeNum == 0:
        return
    
    feature_ConfigToVal = training_data_summary( trainRDD )
    
    interNode_cnt=0
    non_leaf_cnt=0
    recog_config_cnt=0
    
    qu=[]
    current_depth=0
    qu.append((0,total_featureVal_set,0))
    
    
    recog_config_cnt_depth=[]
    recog_conf=[]
    
    #debug
    test_queue=[]
    
    while current_depth<= depth:
        
        print 'non_leaf_node   count at previous depth', non_leaf_cnt
        
        print 'depth', current_depth,':', len(qu)
        tmpqu=[]
        
        non_leaf_cnt=0
        
        for i in qu:
            current_nodeIdx= i[0] 
            current_featureVal_set= i[1]
            
            split_feature= tree[current_nodeIdx][0]
            split_valueSet= tree[current_nodeIdx][1]
                 
            if split_feature == -1: 
                
                if config_check( current_featureVal_set, numFeatures) ==1:
                    recog_config_cnt= recog_config_cnt+1
                    
                    tmpval=lookup_featureConfig_val( feature_ConfigToVal, current_featureVal_set, featureNum)
                    
                    print current_nodeIdx,current_featureVal_set, 'has the estimate:',\
                    tree_esti[ current_nodeIdx*2+1],'true:',tmpval
                    #result
                    recog_conf.append( (tree_esti[ current_nodeIdx*2+1], tmpval) )
                    
                else:
                    print current_nodeIdx,current_featureVal_set, 'has the estimate:',\
                    tree_esti[ current_nodeIdx*2+1],'!!! not seperated !!!'
                    
#                     print '!!! debug !!!', current_nodeIdx,tree[current_nodeIdx], current_featureVal_set
                
                interNode_cnt=interNode_cnt+1
                continue
                
            else:
                non_leaf_cnt=non_leaf_cnt+1
                
                featureValue_list = feature_value_extract( split_valueSet)
                
                print current_nodeIdx, current_featureVal_set,'non-leaf node,', split_feature, featureValue_list
                
                
            #debug
#             if current_depth == 9:
#                 print '+++ debug ++++',current_nodeIdx,tree[current_nodeIdx], current_featureVal_set, \
#                 featureValue_list,split_feature
    
            left_featureValue_set=[]
            right_featureValue_set=[]
            featureNum= len(current_featureVal_set)
    
            for i in range(0, numFeatures):
                if i != split_feature:
                    left_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
                    right_featureValue_set.append(  copy.deepcopy(current_featureVal_set[i])   )
                else:
                    left_featureValue_set.append(  copy.deepcopy(featureValue_list )   )
                    right_featureValue_set.append(copy.deepcopy(list(set(current_featureVal_set[i] )-set(featureValue_list)))   )
            
            
#             #debug
#             if current_depth==8:
#                 if left_featureValue_set== [[2], [11, 13], [103]] or \
#                 right_featureValue_set==right_featureValue_set:
#                     print '????? debug ????', current_nodeIdx,left_featureValue_set,right_featureValue_set
                
            
            
            
            tmpqu.append(  (current_nodeIdx*2+1,left_featureValue_set, current_nodeIdx)  )
            tmpqu.append(  (current_nodeIdx*2+2,right_featureValue_set,current_nodeIdx) )
        
        qu=copy.deepcopy(tmpqu)
        current_depth= current_depth+1
        
        #result
        recog_config_cnt_depth.append( recog_config_cnt )
    
    
# print out leaf nodes

    print 'non_leaf_node count at previous depth', non_leaf_cnt
    print 'bottom depth', current_depth,':', len(qu)
    
    for i in qu:
        current_nodeIdx= i[0] 
        current_featureVal_set= i[1]
        parent_nodeIdx= i[2]
        
#         debug 
#         print '----debug----', current_featureVal_set
        
        if 2*parent_nodeIdx+1 == current_nodeIdx: 
#         parent_nodeIdx == (current_nodeIdx*2+1):
            tree_esti_val= tree_esti[ parent_nodeIdx*2]
        else:
            tree_esti_val= tree_esti[ parent_nodeIdx*2+1] 
        
        if config_check( current_featureVal_set, featureNum) ==1:
            recog_config_cnt= recog_config_cnt+1
            tmpval=lookup_featureConfig_val( feature_ConfigToVal, current_featureVal_set, featureNum)
            
            #result
            recog_conf.append( (tree_esti_val, tmpval) )
            
            print current_nodeIdx,current_featureVal_set, 'has the estimate:', tree_esti_val,\
                    'true:',tmpval
        else:
            print current_nodeIdx,current_featureVal_set, 'has the estimate:', tree_esti_val
    
    #result
    recog_config_cnt_depth.append( recog_config_cnt )
        
    print 'inter-node number:', interNode_cnt
    print ' *******  number of identified configurations:', recog_config_cnt
    
    tmpval=0
    for i in range(0,recog_config_cnt):
        tmpval=tmpval+ (recog_conf[i][0]-recog_conf[i][1])*(recog_conf[i][0]-recog_conf[i][1])
    
    return (recog_conf, recog_config_cnt_depth, tmpval*1.0/recog_config_cnt)

# len( tree_history_split)
# len(tree_history_esti)


In [96]:
# write leaf-nodes results to txt file
text_file = open("regTree_binNumFixed.txt", 'a')
text_file.write('\n \nleaf nodes at depth ')
text_file.write("%f: \n" % maxdepth)
for item in leaf_nodes:
    text_file.write("%f  " % item)

text_file.close()

In [ ]:
# # training and testing data preparation
# # sample from the original data RDD variable: dta

# # outlier threshold to filter outliers in the testing dataset
# outlier_threshold = 40000

# print dta.count()


# dta_train_all = dta.cache().sample(False, .12, 12345)
# dta_test_all = dta.cache().sample(False, .1, 43243)

# print dta_train_all.count(), dta_train_all.getNumPartitions()
# print dta_test_all.count(),  dta_test_all.getNumPartitions()

# # training data set can be set to have outerliers or not
# dta_train = dta_train_all
# # .filter(lambda line: line[0]<outlier_threshold )

# # tmprdd= dta_train.filter(lambda line: line[0]<outlier_threshold )
# tmprdd = dta_train.filter(lambda line: line[0]<outlier_threshold )
# ymax = tmprdd.map(lambda line: line[0]).max()
# ymin = tmprdd.map(lambda line: line[0]).min()
# print ymax, ymin

# # print dta_train.map(lambda line: line[0]).take(10)
# # print dta_train.map(lambda line: line[0]).min()

# # filter outliers for testing data
# dta_test = dta_test_all.filter(lambda line: line[0]< outlier_threshold )
# # ymax = dta_test.map(lambda line: line[0]).max()
# # ymin = dta_test.map(lambda line: line[0]).min()
# # print ymax, ymin

# print 'number of training data instance:',dta_train.count()
# print 'number of testing data instance:',dta_test.count()
# print dta_train.first()